In [3]:
import os
import autogen
from textwrap import dedent
from finrobot.utils import register_keys_from_json
from finrobot.agents.workflow import SingleAssistantShadow

In [4]:
llm_config = {
    "config_list": autogen.config_list_from_json(
        "../OAI_CONFIG_LIST",
        filter_dict={
            "model": ["gpt-4o-mini"],
        },
    ),
    "timeout": 120,
    "temperature": 0.5,
}
register_keys_from_json("../config_api_keys")

In [5]:
# Intermediate results will be saved in this directory
work_dir = "../report"
os.makedirs(work_dir, exist_ok=True)

assistant = SingleAssistantShadow(
    "Expert_Investor",
    llm_config,
    max_consecutive_auto_reply=None,
    human_input_mode="TERMINATE",
)

In [6]:
company = "QCOM"
fyear = "2024"

text =     f"""
    With the files in directory you've been provided, retrieve {company}'s {fyear} files and analyze financial data.
    Pay attention to the followings:
    - Do not compile annual report.
    - Explicitly explain your working plan before you kick off.
    - Use tools one by one for clarity, especially when asking for instructions. 
    - All your file operations should be done in "{work_dir}". 
    - Display any image in the chat once generated.
    - Terminate the conversation when you're done.
"""
message = dedent(text
)

assistant.chat(message, use_cache=True, max_turns=50,
               summary_method="last_msg")

User_Proxy (to Expert_Investor):


With the files in directory you've been provided, retrieve QCOM's 2024 files and analyze financial data.
Pay attention to the followings:
- Do not compile annual report.
- Explicitly explain your working plan before you kick off.
- Use tools one by one for clarity, especially when asking for instructions. 
- All your file operations should be done in "../report". 
- Display any image in the chat once generated.
- Terminate the conversation when you're done.


--------------------------------------------------------------------------------
Expert_Investor (to User_Proxy):

To analyze Qualcomm's (QCOM) financial data for the year 2024, I will follow this structured working plan:

### Working Plan

1. **Retrieve the 10-K Report**: First, I will obtain the filing date and URL of Qualcomm's 10-K report for the year 2024. This report contains comprehensive financial information.

2. **Analyze Financial Statements**:
   - **Income Statement**: I will extract

KeyboardInterrupt: 

In [ ]:
from openai import OpenAI
import os
from textwrap import dedent

client = OpenAI(api_key="", base_url="https://api.openai.com:10443/v1")

work_dir = "../report"
summaries = []

# Read the content of each file in the specified directory and summarize them one by one
for filename in os.listdir(work_dir):
    file_path = os.path.join(work_dir, filename)
    if os.path.isfile(file_path) and not filename.lower().endswith('.png'):
        with open(file_path, 'r') as file:
            file_content = file.read()
        
        # Use the OpenAI API to generate a summary of the content
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": dedent(
                    """
                    Role: Expert Investor
                    Department: Finance
                    Primary Responsibility: Summarization of Customized Financial Analysis Documents

                    Role Description:
                    As an Expert Investor within the finance domain, your expertise is harnessed to summarise Financial Analysis documents that cater to specific client requirements. This role demands a deep dive into financial statements and market data to unearth insights regarding a company's financial performance and stability. Engaging directly with clients to gather essential information and continuously refining the report with their feedback ensures the final product precisely meets their needs and expectations.

                    Key Objectives:

                    Analytical Precision: Employ meticulous analytical prowess to interpret financial data, identifying underlying trends and anomalies.
                    Effective Communication: Simplify and effectively convey complex financial narratives, making them accessible and actionable to non-specialist audiences.
                    Client Focus: Dynamically tailor reports in response to client feedback, ensuring the final analysis aligns with their strategic objectives.
                    Adherence to Excellence: Maintain the highest standards of quality and integrity in report generation, following established benchmarks for analytical rigor.
                    Performance Indicators:
                    The efficacy of the Financial Analysis Summarization is measured by its utility in providing clear, actionable insights. This encompasses aiding corporate decision-making, pinpointing areas for operational enhancement, and offering a lucid evaluation of the company's financial health. Success is ultimately reflected in the report's contribution to informed investment decisions and strategic planning.

                    Reply TERMINATE when everything is settled.
                    """)},
                {"role": "user", "content": f"Summarize the following content. The length of the summary should be around 120 words:\n\n{file_content}"},
            ],
        )

        summary = completion.choices[0].message.content
        summaries.append(f"Summary for {filename}:\n{summary}\n")

# Combine all summaries into a single string
combined_summaries = "\n".join(summaries)

# Write the combined summaries to a single text file
output_file_path = "combined_summaries.txt"
with open(output_file_path, 'w') as output_file:
    output_file.write(combined_summaries)

title_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful expert investor."},
        {"role": "user", "content": f"Generate a title (about 20 words) for the following summary for the stock {company}:\n\n{combined_summaries}"},
    ],
)

title = title_completion.choices[0].message.content

with open(output_file_path, 'a') as output_file:
    output_file.write(f"\nTitle: {title}\n")

print(f"Summaries and title have been written to {output_file_path}")
print(f"Title: {title}")